In [1]:
import requests
from bs4 import BeautifulSoup

def increment_month(ym):
    """
    YYYYMM形式の整数を受け取り、1ヶ月進めた値を返す。
    例：202012 -> 202101, 202101 -> 202102
    """
    year = ym // 100
    month = ym % 100
    month += 1
    if month > 12:
        month = 1
        year += 1
    return year * 100 + month

def scrape_event_detail(detail_url):
    response = requests.get(detail_url)
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, 'html.parser')
    
    event_data = {}
    
    # タイトル
    title_tag = soup.select_one('.eventDetail .eventTit')
    event_data['detail_title'] = title_tag.get_text(strip=True) if title_tag else None
    
    # カテゴリ
    category_tag = soup.select_one('.eventDetail .category')
    event_data['detail_category'] = category_tag.get_text(strip=True) if category_tag else None
    
    # 説明文
    disc_tag = soup.select_one('.eventDetail .disc')
    event_data['description'] = disc_tag.get_text(strip=True) if disc_tag else None
    
    # 開催期間
    date_dd = soup.select_one('.eventDetail dl.date dd')
    if date_dd:
        date_main = date_dd.select_one('.main')
        event_data['date_period'] = date_main.get_text(strip=True) if date_main else None

    # 開催時間
    time_dd = soup.select_one('.eventDetail dl.time dd')
    if time_dd:
        time_main = time_dd.select_one('.main')
        time_note = time_dd.select_one('.note')
        event_data['time_main'] = time_main.get_text(strip=True) if time_main else None
        event_data['time_note'] = time_note.get_text(strip=True) if time_note else None

    # 対象者/入場料
    person_dd = soup.select_one('.eventDetail dl.person dd .authorP')
    if person_dd:
        target_div = person_dd.select_one('.target')
        price_div = person_dd.select_one('.price')
        event_data['detail_target'] = target_div.get_text(strip=True) if target_div else None
        event_data['detail_price'] = price_div.get_text(strip=True) if price_div else None

    # 利用施設
    facility_dd = soup.select('.eventDetail dl.facility dd .hall')
    if facility_dd:
        facilities = []
        for hall in facility_dd:
            hall_name = hall.select_one('.hallName')
            hall_no = hall.select_one('.hallNo')
            facility_str = ''
            if hall_name:
                facility_str += hall_name.get_text(strip=True)
            if hall_no:
                facility_str += ' ' + hall_no.get_text(strip=True)
            facilities.append(facility_str.strip())
        event_data['facilities'] = facilities

    # 主催者
    organizer_dd = soup.select_one('.eventDetail dl.organizer dd')
    event_data['organizer'] = organizer_dd.get_text(strip=True) if organizer_dd else None

    # URL
    url_dd = soup.select_one('.eventDetail dl.contact dd .url a')
    event_data['official_url'] = url_dd.get('href') if url_dd else None

    # 詳細ページ画像
    thumb_img = soup.select_one('.eventDetail .thumb img')
    if thumb_img:
        img_src = thumb_img.get('src')
        if img_src and img_src.startswith('/'):
            img_src = 'https://www.m-messe.co.jp' + img_src
        event_data['detail_image_url'] = img_src
    else:
        event_data['detail_image_url'] = None
    
    return event_data


def scrape_events_for_month(month, pmonth, ss, max_pages=100):
    """
    指定のmonth, pmonth, ssでイベント一覧と詳細情報を取得。
    max_pagesは1ヶ月あたりに取得を試みるページ数の上限。
    イベントが取得できなくなったらbreak。
    """
    all_events = []
    for page in range(1, max_pages+1):
        url = "https://www.m-messe.co.jp/event/"
        params = {
            "month": str(month),
            "c": "",
            "word": "",
            "page": str(page),
            "pmonth": str(pmonth),
            "ss": str(ss),
        }
        response = requests.get(url, params=params)
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, "html.parser")

        event_list = soup.select("ul.eventList > li.eventInr.clear")
        if not event_list:
            # イベントがなければbreak
            break

        for li in event_list:
            a_tag = li.find("a")
            if a_tag:
                detail_url = a_tag.get("href")
                if detail_url and detail_url.startswith("/event/detail/"):
                    detail_url = "https://www.m-messe.co.jp" + detail_url
                else:
                    detail_url = None
            else:
                detail_url = None

            category_div = li.select_one(".category")
            category = category_div.get_text(strip=True) if category_div else None

            date_div = li.select_one(".date")
            date_text = date_div.get_text(strip=True) if date_div else None

            title_div = li.select_one(".eventTit")
            title = title_div.get_text(strip=True) if title_div else None

            target_div = li.select_one(".target")
            target_text = target_div.get_text(strip=True).replace("対象", "") if target_div else None

            thumb_div = li.select_one(".thumb img")
            image_url = thumb_div.get("src") if thumb_div else None
            if image_url and image_url.startswith("/"):
                image_url = "https://www.m-messe.co.jp" + image_url

            event = {
                "title": title,
                "date": date_text,
                "category": category,
                "target": target_text,
                "detail_url": detail_url,
                "image_url": image_url,
                "month_param": month
            }
            
            # detail_urlがあれば詳細情報取得
            if detail_url:
                detail_data = scrape_event_detail(detail_url)
                event.update(detail_data)
            
            all_events.append(event)

    return all_events

def scrape_all_events(start_month=202012, end_month=202606):
    """
    start_monthからend_monthまで月を進めながらイベントを取得。
    ssはstart_monthから進めるたびに+1していく。
    pmonthは固定値としてend_monthを渡します。
    """
    all_events = []
    ss = 0
    pmonth = end_month

    current_month = start_month
    while current_month <= end_month:
        events = scrape_events_for_month(current_month, pmonth, ss)
        all_events.extend(events)

        # 次の月へ
        current_month = increment_month(current_month)
        ss += 1

    return all_events


# 例: 2020年12月から2021年6月までのイベントを取得
data = scrape_all_events(start_month=202012, end_month=202606)


In [2]:
data[-1]

{'title': '通信制高校・サポート校合同相談会',
 'date': '2025.11.24(月)',
 'category': 'イベント',
 'target': '一般',
 'detail_url': 'https://www.m-messe.co.jp/event/detail/7921',
 'image_url': 'https://www.m-messe.co.jp/saved/images/event/8e/63/c5f31288126de5d09a023f5150e8ebe514bf8e63.jpg',
 'month_param': 202511,
 'detail_title': '通信制高校・サポート校合同相談会',
 'detail_category': 'イベント',
 'description': '通信制高校、サポート校、高等専修学校、技能連携校等の多様な進路についての合同相談会です。各校ブースで話を聴けるほか、通信制のしくみに関する講演や在校生・卒業生の体験談、各種相談コーナーもあります。',
 'date_period': '2025年11月24日(月・祝)',
 'time_main': '10:30～15:30',
 'time_note': None,
 'facilities': ['国際会議場'],
 'organizer': '学びリンク株式会社',
 'official_url': 'https://www.stepup-school.net/soudankai/',
 'detail_image_url': 'https://www.m-messe.co.jp/cache/images/event/8e/63/c5f31288126de5d09a023f5150e8ebe514bf8e63.1200x1200.none.jpg'}

In [3]:
import pandas as pd

# 辞書型データをDataFrameに変換
df = pd.DataFrame(data)



In [4]:
df[df['month_param']==202410]

,title,date,category,target,detail_url,image_url,month_param,detail_title,detail_category,description,date_period,time_main,time_note,detail_target,detail_price,facilities,organizer,official_url,detail_image_url
935,第3回 サーキュラー・エコノミー EXPO【秋】 [CE JAPAN],2024.10.02(水) 〜 2024.10.04(金),展示会・見本市,関係者,https://www.m-messe.co.jp/event/detail/7639,https://www.m-messe.co.jp/saved/images/event/a...,202410,第3回 サーキュラー・エコノミー EXPO【秋】 [CE JAPAN],展示会・見本市,サーキュラーデザイン、サステナブルマテリアル、PaaS支援、資源回収・リサイクル・再製品化技...,2024年10月2日(水)～2024年10月4日(金),10:00～17:00,None,関係者,無料(要登録),[国際展示場 展示ホール5〜8],RX Japan株式会社,https://www.decarbonization-expo.jp/hub/ja-jp/...,https://www.m-messe.co.jp/cache/images/event/a...
936,GX経営 WEEK【秋】2024,2024.10.02(水) 〜 2024.10.04(金),展示会・見本市,関係者,https://www.m-messe.co.jp/event/detail/7619,https://www.m-messe.co.jp/saved/images/event/0...,202410,GX経営 WEEK【秋】2024,展示会・見本市,コーポレートPPAや再エネ電力、エネマネ技術、ZEB/スマートビルや次世代空調などの企業向け...,2024年10月2日(水)～2024年10月4日(金),10:00～17:00,None,関係者,無料(要登録),[国際展示場 展示ホール5〜8],RX Japan株式会社,https://www.decarbonization-expo.jp/autumn/ja-...,https://www.m-messe.co.jp/cache/images/event/0...
937,SMART ENERGY WEEK ～スマートエネルギー WEEK～【秋】2024,2024.10.02(水) 〜 2024.10.04(金),展示会・見本市,関係者,https://www.m-messe.co.jp/event/detail/7562,https://www.m-messe.co.jp/saved/images/event/b...,202410,SMART ENERGY WEEK ～スマートエネルギー WEEK～【秋】2024,展示会・見本市,世界中からエネルギー分野500社が出展。太陽光発電・風力発電・水素エネルギー・スマートグリッ...,2024年10月2日(水)～2024年10月4日(金),10:00～17:00,None,関係者,無料(要登録),[国際展示場 展示ホール5〜8],RX Japan株式会社,https://www.wsew.jp/autumn/ja-jp.html,https://www.m-messe.co.jp/cache/images/event/b...
938,BATTERY JAPAN【秋】～第17回 [国際] 二次電池展～,2024.10.02(水) 〜 2024.10.04(金),展示会・見本市,関係者,https://www.m-messe.co.jp/event/detail/7567,https://www.m-messe.co.jp/saved/images/event/4...,202410,BATTERY JAPAN【秋】～第17回 [国際] 二次電池展～,展示会・見本市,世界中から部品・材料、材料開発支援、評価・測定・検査、製造装置・付帯設備、バッテリーマネジメ...,2024年10月2日(水)～2024年10月4日(金),10:00～17:00,None,関係者,無料(要登録),[国際展示場 展示ホール5〜8],RX Japan株式会社,https://www.wsew.jp/hub/ja-jp/about/bj.html,https://www.m-messe.co.jp/cache/images/event/4...
939,SMART GRID EXPO【秋】～第16回 [国際] スマートグリッド展～,2024.10.02(水) 〜 2024.10.04(金),展示会・見本市,関係者,https://www.m-messe.co.jp/event/detail/7568,https://www.m-messe.co.jp/saved/images/event/0...,202410,SMART GRID EXPO【秋】～第16回 [国際] スマートグリッド展～,展示会・見本市,世界中からエネルギーマネジメント、蓄電システム、VPP関連技術、電力設備などが出展。技術的な...,2024年10月2日(水)～2024年10月4日(金),10:00～17:00,None,関係者,無料(要登録),[国際展示場 展示ホール5〜8],RX Japan株式会社,https://www.wsew.jp/hub/ja-jp/about/sg.html,https://www.m-messe.co.jp/cache/images/event/0...
940,WIND EXPO【秋】～第14回 [国際] 風力発電展～,2024.10.02(水) 〜 2024.10.04(金),展示会・見本市,関係者,https://www.m-messe.co.jp/event/detail/7569,https://www.m-messe.co.jp/saved/images/event/c...,202410,WIND EXPO【秋】～第14回 [国際] 風力発電展～,展示会・見本市,世界中から風力発電関連の部品・駆動装置、施工・輸送、解析・計測、洋上風量技術、メンテナンス会...,2024年10月2日(水)～2024年10月4日(金),10:00～17:00,None,関係者,無料(要登録),[国際展示場 展示ホール5〜8],RX Japan株式会社,https://www.wsew.jp/hub/ja-jp/about/wd.html,https://www.m-messe.co.jp/cache/images/event/c...
941,H2 & FC EXPO【秋】～第22回 [国際] 水素・燃料電池展～,2024.10.02(水) 〜 2024.10.04(金),展示会・見本市,関係者,https://www.m-messe.co.jp/event/detail/7565,https://www.m-messe.co.jp/saved/images/event/f...,202410,H2 & FC EXPO【秋】～第22回 [国際] 水素・燃料電池展～,展示会・見本市,世界中から水素製造・貯蔵・供給技術、評価・測定・分析機器、燃料電池部品・材料、燃料電池製品・...,2024年10月2日(水)～2024年10月4日(金),10:00～17:00,None,関係者,無料(要登録),[国際展示場 展示ホール5〜8],RX Japan 株式会社,https://www.wsew.jp/hub/ja-jp/about/fc.html,https://www.m-messe.co.jp/cache/images/event/f...
942,PV EXPO【秋】～第19回 [国際] 太陽光発電展～,2024.10.02(水) 〜 2024.10.04(金),展示会・見本市,関係者,https://www.m-messe.co.jp/event/detail/7566,https://www.m-messe.co.jp/saved/images/event/4...,202410,PV EXPO【秋】～第19回 [国際] 太陽光発電展～,展示会・見本市,世界中から太陽光パネル、施工用資材、O&M技術、周辺機器(パワコン、ケーブルなど)が出展。技...,2024年10月2日(水)～2024年10月4日(金),10:00～17:00,None,関係者,無料(要登録),[国際展示場 展示ホール5〜8],RX Japan 株式会社,https://www.wsew.jp/hub/ja-jp/about/pv.html,https://www.m-messe.co.jp/cache/images/event/4...
943,第5回 脱炭素経営 EXPO【秋】,2024.10.02(水) 〜 2024.10.04(金),展示会・見本市,関係者,https://www.m-messe.co.jp/event/detail/7570,https://www.m-messe.co.jp/saved/images/event/6...,202410

In [5]:
# DataFrameをCSVファイルとして保存
df.to_csv("events.csv", index=False, encoding="utf-8")

In [ ]:
#CSVデータをevent_detailsTBLへ変換する

In [6]:
import csv
import re
import datetime

def parse_time_range(time_str: str):
    """
    例）"10:00～18:00" から
        open_time="10:00", close_time="18:00"
    を抽出する簡易パーサ。
    """
    pattern = r'(\d{1,2}:\d{2})\D+(\d{1,2}:\d{2})'
    match = re.search(pattern, time_str)
    if match:
        start = match.group(1)
        end = match.group(2)
        return start, end
    else:
        return "", ""

def parse_price(detail_price: str):
    """
    例）"有料\\5,000/人(招待券持参者無料)" から
         max_price=5000, min_price=0
    を抽出する簡易パーサ。
    """
    # 数字(0-9)をすべて拾ってリスト化
    # カンマや円マーク(\)があっても一旦除去して抽出
    numbers = re.findall(r'\d+', detail_price.replace(',', ''))
    if numbers:
        numeric_values = list(map(int, numbers))
        max_val = max(numeric_values)
        min_val = min(numeric_values)
    else:
        max_val = 0
        min_val = 0
    
    # 「無料」という文字列があれば最小価格を0とみなす
    if "無料" in detail_price:
        min_val = 0

    return max_val, min_val

def clean_facilities_str(facilities_str: str):
    """
    例）"['国際展示場 展示ホール1〜6']" のように
         リスト形式っぽくなっている場合は
         先頭の "['" と末尾の "']" を除去するなどして
         普通の文字列にする。
    """
    cleaned = facilities_str.strip()
    # 先頭の "['" や末尾の "']" を削除する
    cleaned = re.sub(r"^\[\'+", "", cleaned)
    cleaned = re.sub(r"\'\]$", "", cleaned)
    return cleaned

def parse_jp_date(date_str: str):
    """
    "YYYY年MM月DD日" 形式の日付文字列をパースして
    datetime.date を返す。
    例）"2020年12月2日(水)" -> date(2020,12,2)
    """
    pattern = r'(\d{4})年(\d{1,2})月(\d{1,2})日'
    match = re.search(pattern, date_str)
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        day = int(match.group(3))
        return datetime.date(year, month, day)
    return None

def expand_date_range(date_period_str: str):
    """
    "2020年12月2日(水)～2020年12月4日(金)" のような文字列を解析し、
    開始日から終了日までを "YYYY-MM-DD" のリストにして返す。
    ※ 開始日や終了日がパースできなかった場合は空リストを返す。
    """
    # "～" で分割して前半を開始日、後半を終了日とみなす
    splitted = date_period_str.split('～')
    if len(splitted) == 2:
        start_str = splitted[0].strip()
        end_str   = splitted[1].strip()

        start_date = parse_jp_date(start_str)  # 例）2020-12-02
        end_date   = parse_jp_date(end_str)    # 例）2020-12-04

        if start_date and end_date:
            result_dates = []
            delta_days = (end_date - start_date).days
            for i in range(delta_days + 1):
                d = start_date + datetime.timedelta(days=i)
                result_dates.append(d.strftime("%Y-%m-%d"))
            return result_dates

    # パースできない場合は空リスト
    return []

def main_transform(input_csv: str, output_csv: str):
    """
    取得したCSV(input_csv)を読み込み、
    変換後のCSV(output_csv)を作成する。
    ここで 'date_period' から
    複数の 'date' 行を展開して出力。
    """
    output_header = [
        "id",
        "name",
        "sub_title",
        "place",
        "actor",
        "date",
        "open_time",
        "start_time",
        "close_time",
        "time_original",
        "max_price",
        "min_price",
        "price_text",
        "link",
        "inquiry",
        "detail",
        "num_of_people",
        "input_table",
        "input_table_rec_id",
    ]
    
    with open(input_csv, mode="r", encoding="utf-8") as fin, \
         open(output_csv, mode="w", encoding="utf-8", newline="") as fout:
        
        reader = csv.DictReader(fin)
        writer = csv.writer(fout)
        
        # 変換後CSVのヘッダーを書き込み
        writer.writerow(output_header)
        
        row_id = 0  # id（複数行展開後も連番を振る）
        
        for row in reader:
            # (1) 元CSVの値を取得
            title        = row.get("title", "")
            sub_title    = row.get("category", "")
            place        = clean_facilities_str(row.get("facilities", ""))
            actor        = row.get("organizer", "")
            date_period  = row.get("date_period", "")
            time_main    = row.get("time_main", "")
            time_note    = row.get("time_note", "")
            detail_price = row.get("detail_price", "")
            link         = row.get("official_url", "")
            inquiry      = row.get("detail_url", "")
            detail       = row.get("description", "")
            
            # (2) 時間関連のパース
            open_t, close_t = parse_time_range(time_main)
            start_t = open_t  # 今回は start_time=open_time とする例
            time_original = time_main
            if time_note:
                time_original += f"（{time_note}）"
            
            # (3) 料金のパース
            max_price, min_price = parse_price(detail_price)
            
            # (4) 日付範囲を展開
            #  例）"2020年12月2日(水)～2020年12月4日(金)" -> ["2020-12-02", "2020-12-03", "2020-12-04"]
            date_list = expand_date_range(date_period)
            if not date_list:
                # 日付が取得できなかった場合、1行だけ出力する（空のdate）
                date_list = [""]
            
            # (5) 日付の数だけループし、それぞれ1行ずつ出力
            for one_date in date_list:
                row_id += 1
                
                output_row = [
                    row_id,          # id(連番)
                    title,           # name
                    sub_title,       # sub_title
                    place,           # place
                    actor,           # actor
                    one_date,        # date(YYYY-MM-DD)
                    open_t,          # open_time
                    start_t,         # start_time
                    close_t,         # close_time
                    time_original,   # time_original
                    max_price,       # max_price
                    min_price,       # min_price
                    detail_price,    # price_text
                    link,            # link
                    inquiry,         # inquiry
                    detail,          # detail
                    "",              # num_of_people(今回は空文字)
                    "csv",           # input_table(固定値)
                    row_id,          # input_table_rec_id(ここではidと同一を採用)
                ]
                
                writer.writerow(output_row)

    print(f"変換が完了しました。出力ファイル: {output_csv}")

if __name__ == "__main__":
    # 例として input.csv -> output.csv への変換を実行
    main_transform("events.csv", "output.csv")

変換が完了しました。出力ファイル: output.csv


In [7]:
df = pd.read_csv("output.csv")

In [10]:
df[df['date']=="2025-01-20"]

,id,name,sub_title,place,actor,date,open_time,start_time,close_time,time_original,max_price,min_price,price_text,link,inquiry,detail,num_of_people,input_table,input_table_rec_id


KeyError: 'month'